<a href="https://colab.research.google.com/github/nelsonufomba/Intro-to-CS50-Lab-1-Python/blob/main/ECE_57000_COSAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import argparse
import pickle
import time
import numpy as np
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
import pandas as pd
import csv
import operator
import datetime
import os
from torch.utils.data import Dataset


In [ ]:
print(f'Can I can use GPU now? -- {torch.cuda.is_available()}')

rand_tensor = torch.rand(5,2)
simple_model = nn.Sequential(nn.Linear(2,10), nn.ReLU(), nn.Linear(10,1))
print(f'input is on {rand_tensor.device}')
print(f'model parameters are on {[param.device for param in simple_model.parameters()]}')
print(f'output is on {simple_model(rand_tensor).device}')

# device = torch.device('cuda')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# ----------- <Your code> ---------------
# Move rand_tensor and model onto the GPU device
rand_tensor = rand_tensor.to(device)
simple_model = simple_model.to(device)

# --------- <End your code> -------------
print(f'input is on {rand_tensor.device}')
print(f'model parameters are on {[param.device for param in simple_model.parameters()]}')
print(f'output is on {simple_model(rand_tensor).device}')

Can I can use GPU now? -- True
input is on cpu
model parameters are on [device(type='cpu'), device(type='cpu'), device(type='cpu'), device(type='cpu')]
output is on cpu
input is on cuda:0
model parameters are on [device(type='cuda', index=0), device(type='cuda', index=0), device(type='cuda', index=0), device(type='cuda', index=0)]
output is on cuda:0


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# Define the path to your data file
file_path = '/content/drive/MyDrive/ECE57000Project/archive-3/yoochoose-clicks.dat'

# Load the data into a DataFrame and add headers if not already present
headers = ['session_id', 'timestamp', 'item_id', 'category']
data = pd.read_csv(file_path, header=None, names=headers)
#data.to_csv(file_path, index=False)  # Save with headers if needed

# Dataset and file path setup
dataset_option = 'yoochoose'  # Choose 'sample' or 'yoochoose'
dataset_file = file_path

# Display the first 5 rows
print(data.head(5))

print("-- Starting @ %s" % datetime.datetime.now())


<ipython-input-6-d722d8bd75e0>:6: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path, header=None, names=headers)


   session_id                 timestamp    item_id category
0           1  2014-04-07T10:51:09.277Z  214536502        0
1           1  2014-04-07T10:54:09.868Z  214536500        0
2           1  2014-04-07T10:54:46.998Z  214536506        0
3           1  2014-04-07T10:57:00.306Z  214577561        0
4           2  2014-04-07T13:56:37.614Z  214662742        0
-- Starting @ 2024-11-07 01:26:27.599515


In [ ]:
# Drop the first three rows
data = data.drop(index=[0]).reset_index(drop=True)
data.to_csv(file_path, index=False)  # Save with headers if needed

# Display the first 5 rows after dropping
print(data.head(5))

  session_id                 timestamp    item_id category
0          1  2014-04-07T10:51:09.277Z  214536502        0
1          1  2014-04-07T10:54:09.868Z  214536500        0
2          1  2014-04-07T10:54:46.998Z  214536506        0
3          1  2014-04-07T10:57:00.306Z  214577561        0
4          2  2014-04-07T13:56:37.614Z  214662742        0


In [ ]:
import csv
import os
from pathlib import Path
import pickle
from datetime import datetime
from collections import defaultdict
import operator

# Constants
MIN_ITEM_OCCURRENCES = 5
MIN_SESSION_LENGTH = 2
SECONDS_PER_DAY = 86400
YOOCHOOSE_SPLIT_DAYS = 1
DIGINETICA_SPLIT_DAYS = 7

def read_dataset(file_path):
    sess_clicks = defaultdict(list)
    sess_date = {}

    try:
        with open(file_path, "r") as f:
            reader = csv.reader(f, delimiter=',')
            next(reader)  # Skip header row

            for row in reader:
                session_id, timestamp, item_id = row[0], row[1], row[2]

                # Convert timestamp to UNIX timestamp
                unix_time = parse_timestamp(timestamp)

                sess_clicks[session_id].append(item_id)
                sess_date[session_id] = unix_time

        print(f"Data read successfully at {datetime.now()}")
        return sess_clicks, sess_date
    except Exception as e:
        print(f"Error reading dataset: {e}")
        return None, None

def parse_timestamp(timestamp):
    if timestamp.endswith('Z'):
        timestamp = timestamp[:-1]  # Remove 'Z' for parsing
    return datetime.strptime(timestamp, '%Y-%m-%dT%H:%M:%S.%f').timestamp()

def filter_sessions(sess_clicks, sess_date):
    # Filter sessions with only one click
    sess_clicks = {s: clicks for s, clicks in sess_clicks.items() if len(clicks) > 1}
    sess_date = {s: sess_date[s] for s in sess_clicks}

    # Count item occurrences
    iid_counts = defaultdict(int)
    for seq in sess_clicks.values():
        for iid in seq:
            iid_counts[iid] += 1

    # Filter items appearing fewer than MIN_ITEM_OCCURRENCES times
    sess_clicks = {
        s: [i for i in seq if iid_counts[i] >= MIN_ITEM_OCCURRENCES]
        for s, seq in sess_clicks.items()
        if len([i for i in seq if iid_counts[i] >= MIN_ITEM_OCCURRENCES]) >= MIN_SESSION_LENGTH
    }

    return sess_clicks, sess_date

def split_data(sess_clicks, sess_date, dataset_option):
    split_days = YOOCHOOSE_SPLIT_DAYS if dataset_option == 'yoochoose' else DIGINETICA_SPLIT_DAYS
    split_date = max(sess_date.values()) - (SECONDS_PER_DAY * split_days)

    tra_sess = sorted([(s, d) for s, d in sess_date.items() if d < split_date], key=operator.itemgetter(1))
    tes_sess = sorted([(s, d) for s, d in sess_date.items() if d >= split_date], key=operator.itemgetter(1))

    print(f"Training sessions: {len(tra_sess)}, Testing sessions: {len(tes_sess)}")
    return tra_sess, tes_sess

def obtain_sequences(sessions, sess_clicks, item_dict=None):
    seq_ids, seq_dates, seq_items = [], [], []
    item_count = 1 if item_dict is None else len(item_dict) + 1
    sess_clicks = defaultdict(list, sess_clicks)  # Convert to defaultdict

    for session_id, date in sessions:
        outseq = []
        for item in sess_clicks[session_id]:  # Now safe to access
            if item_dict is not None:
                if item not in item_dict:
                    item_dict[item] = item_count
                    item_count += 1
                outseq.append(item_dict[item])
            else:
                outseq.append(item)

        if len(outseq) >= MIN_SESSION_LENGTH:
            seq_ids.append(session_id)
            seq_dates.append(date)
            seq_items.append(outseq)

    return seq_ids, seq_dates, seq_items, item_dict

def save_processed_data(output_dir, train_data, test_data):
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)

    with open(output_path / 'train.txt', 'wb') as f:
        pickle.dump(train_data, f)
    with open(output_path / 'test.txt', 'wb') as f:
        pickle.dump(test_data, f)

    print("Data saved successfully.")

def main(dataset_file, dataset_option):
    # Read and process data
    sess_clicks, sess_date = read_dataset(dataset_file)
    if sess_clicks is None or sess_date is None:
        return

    sess_clicks, sess_date = filter_sessions(sess_clicks, sess_date)
    tra_sess, tes_sess = split_data(sess_clicks, sess_date, dataset_option)

    # Obtain sequences
    train_ids, train_dates, train_seqs, item_dict = obtain_sequences(tra_sess, sess_clicks)
    test_ids, test_dates, test_seqs, _ = obtain_sequences(tes_sess, sess_clicks, item_dict)

    # Confirm sample sequences
    print("Sample train sequences:", train_seqs[:3])
    print("Sample test sequences:", test_seqs[:3])

    # Save processed data
    output_dir = f'/content/drive/MyDrive/ECE57000Project/{("yoochoose1_64" if dataset_option == "yoochoose" else "diginetica")}'
    save_processed_data(output_dir, (train_seqs, train_ids), (test_seqs, test_ids))

    print("Processing Complete.")

if __name__ == "__main__":
    dataset_file = "/content/drive/MyDrive/ECE57000Project/archive-3/yoochoose-clicks.dat"  # Replace with actual path
    dataset_option = "yoochoose"  # or "diginetica"
    main(dataset_file, dataset_option)

Data read successfully at 2024-11-07 00:02:41.958426
Training sessions: 7974678, Testing sessions: 15340
Sample train sequences: [['214594678', '214820231'], ['214687685', '214705119'], ['214716982', '214716982', '214716982']]
Sample test sequences: [['214847885', '214857260', '214602729', '214851120'], ['214858847', '214857257', '214858850', '214859034', '214858850'], ['214717115', '214829820', '214645005']]
Data saved successfully.
Processing Complete.


In [ ]:
# Load processed data
output_dir = '/content/drive/MyDrive/ECE57000Project/yoochoose1_64'  # Adjust based on your output directory
with open(f'{output_dir}/train.txt', 'rb') as f:
    train_data = pickle.load(f)

with open(f'{output_dir}/test.txt', 'rb') as f:
    test_data = pickle.load(f)

train_seqs, train_ids = train_data
test_seqs, test_ids = test_data

print("Loaded training sequences:", len(train_seqs))
print("Loaded testing sequences:", len(test_seqs))

Loaded training sequences: 7966257
Loaded testing sequences: 15324


In [ ]:
# Define a custom dataset class
class SessionDataset(Dataset):
    def __init__(self, sequences, targets):
        self.sequences = sequences
        self.targets = targets # Store targets
    # 1. Create an item-to-ID mapping:

        self.item2id = {}
        self.id2item = {}
        for seq in self.sequences:
            for item in seq:
                if item not in self.item2id:
                    self.item2id[item] = len(self.item2id)
                    self.id2item[len(self.id2item)] = item

        # 2. Use the mapping to convert item sequences to ID sequences:
        self.sequences = [[self.item2id[item] for item in seq] for seq in self.sequences]
        #self.targets = self.sequences  # Or load them separately if needed


    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        items = torch.tensor(self.sequences[idx], dtype=torch.long)
        target = torch.tensor(self.targets[idx], dtype=torch.float)  # Adjust for binary or multi-class targets
        return items, targets

# Assuming 'sequences' is your list of item sequences
import random

# Generate random targets (example - adjust as needed)
targets = [[random.randint(0, 1) for _ in seq] for seq in train_seqs]

# Create datasets
train_dataset = SessionDataset(train_seqs, targets)
test_dataset = SessionDataset(test_seqs, targets)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [ ]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset

def load_in_data_batches(train_path, test_path, batch_size):
    # Load training data
    train_data = pd.read_csv(train_path)  # Adjust based on how you saved your data
    test_data = pd.read_csv(test_path)    # Same as above

    # Assuming train_data and test_data have 'item_sequence' and 'target' columns
    train_sequences = train_data['item_sequence'].apply(eval).tolist()  # Convert string representation of lists back to lists
    train_targets = train_data['target'].tolist()

    test_sequences = test_data['item_sequence'].apply(eval).tolist()
    test_targets = test_data['target'].tolist()

    # Create datasets
    train_dataset = SessionDataset(train_sequences, train_targets)
    test_dataset = SessionDataset(test_sequences, test_targets)

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader

In [ ]:
import torch
import torch.nn as nn

class CollaborativeSelfAttentionNetwork(nn.Module):
    def __init__(self, opt, n_node, n_user):
        super(CollaborativeSelfAttentionNetwork, self).__init__()
        self.hidden_size = opt.hiddenSize
        self.n_node = n_node
        self.n_user = n_user
        self.batch_size = opt.batchSize
        self.n_heads = opt.nhead
        self.n_layers = opt.layer
        self.dropout_rate = opt.dropout
        self.alpha = opt.alpha

        # Embeddings
        self.item_embedding = nn.Embedding(self.n_node, self.hidden_size)
        self.session_embedding = nn.Embedding(self.n_user, self.hidden_size)  # Using n_user as proxy for n_sessions

        # Multi-head self-attention with layer normalization
        self.self_attention = nn.MultiheadAttention(self.hidden_size, self.n_heads, dropout=self.dropout_rate)

        # Feed-forward network with dropout
        self.feed_forward = nn.Sequential(
            nn.Linear(self.hidden_size, self.hidden_size * 4),
            nn.ReLU(),
            nn.Dropout(self.dropout_rate),  # Added dropout here for better regularization
            nn.Linear(self.hidden_size * 4, self.hidden_size)
        )

        # Layer normalization applied after feed-forward network
        self.layer_norm1 = nn.LayerNorm(self.hidden_size)
        self.layer_norm2 = nn.LayerNorm(self.hidden_size)

        # Prediction layer with bias initialization
        self.prediction = nn.Linear(self.hidden_size * 2, self.n_node)
        nn.init.xavier_uniform_(self.prediction.weight)  # Xavier initialization for better convergence

        # Loss function and optimizer setup
        self.loss_function = nn.BCEWithLogitsLoss()
        self.optimizer = torch.optim.Adam(self.parameters(), lr=opt.lr, weight_decay=opt.l2)

        # Learning rate scheduler
        self.scheduler = torch.optim.lr_scheduler.StepLR(self.optimizer, step_size=opt.lr_dc_step, gamma=opt.lr_dc)

    def forward(self, items, session_ids, neighborhood_sessions, neighborhood_weights):
        # Collaborative Item Representation
        item_emb = self.item_embedding(items)

        # Improved neighborhood representation calculation
        neighborhood_emb = self.session_embedding(neighborhood_sessions)
        complementary_feature = torch.sum(neighborhood_emb * neighborhood_weights.unsqueeze(-1), dim=1)

        collaborative_item_rep = item_emb + (self.alpha * complementary_feature)

        # Self-attention layers with residual connections and normalization
        for _ in range(self.n_layers):
            attn_output, _ = self.self_attention(collaborative_item_rep.transpose(0, 1), collaborative_item_rep.transpose(0, 1), collaborative_item_rep.transpose(0, 1))
            attn_output = attn_output.transpose(0, 1)
            collaborative_item_rep = self.layer_norm1(collaborative_item_rep + attn_output)  # Residual connection

            ff_output = self.feed_forward(collaborative_item_rep)
            collaborative_item_rep = self.layer_norm2(collaborative_item_rep + ff_output)  # Residual connection

        # Final session representation combining last item representation and session embedding
        session_emb = self.session_embedding(session_ids)

        final_rep = torch.cat([collaborative_item_rep[:, -1, :], session_emb], dim=1)

        # Prediction scores calculation
        scores = self.prediction(final_rep)

        return scores

    def compute_loss(self, scores, targets):
        return self.loss_function(scores, targets)

In [ ]:
def get_neighborhood_data(items, neighborhood_data):
    batch_size, seq_len = items.size()
    return torch.randint(0, 1000, (batch_size, seq_len, 10)), torch.rand(batch_size, seq_len, 10)

In [ ]:
def train_test(model, train_loader, test_loader, user_ids, neighborhood_data, device):
    print('Start training: ', time.strftime("%Y-%m-%d %H:%M:%S"))
    model.train()
    total_loss = 0.0

    for batch_idx, (items, targets) in enumerate(train_loader):
        model.optimizer.zero_grad()

        batch_size = items.size(0)
        session_ids = user_ids[batch_idx * batch_size : (batch_idx + 1) * batch_size]

        # Get neighborhood data for this batch
        neighborhood_sessions, neighborhood_weights = get_neighborhood_data(items, neighborhood_data)

        items = items.to(device)
        targets = targets.to(device)
        session_ids = torch.LongTensor(session_ids).to(device)
        neighborhood_sessions = neighborhood_sessions.to(device)
        neighborhood_weights = neighborhood_weights.to(device)

        scores = model(items, session_ids, neighborhood_sessions, neighborhood_weights)
        loss = model.compute_loss(scores, targets)

        loss.backward()
        model.optimizer.step()
        total_loss += loss.item()

        if batch_idx % 200 == 0:
            print(f'[{batch_idx}/{len(train_loader)}] Loss: {loss.item():.4f}')

    print(f'\tTotal Loss:\t{total_loss:.3f}')

    print('Start predicting: ', time.strftime("%Y-%m-%d %H:%M:%S"))
    model.eval()
    hit, mrr = [], []

    with torch.no_grad():
        for batch_idx, (items, targets) in enumerate(test_loader):
            batch_size = items.size(0)
            session_ids = user_ids[batch_idx * batch_size : (batch_idx + 1) * batch_size]

            # Get neighborhood data for this batch
            neighborhood_sessions, neighborhood_weights = get_neighborhood_data(items, neighborhood_data)

            items = items.to(device)
            targets = targets.to(device)
            session_ids = torch.LongTensor(session_ids).to(device)
            neighborhood_sessions = neighborhood_sessions.to(device)
            neighborhood_weights = neighborhood_weights.to(device)

            scores = model(items, session_ids, neighborhood_sessions, neighborhood_weights)

            sub_scores = scores.topk(20)[1]
            sub_scores = sub_scores.cpu().numpy()
            targets = targets.cpu().numpy()

            for score, target in zip(sub_scores, targets):
                hit.append(np.isin(target - 1, score))
                if target - 1 in score:
                    mrr.append(1 / (np.where(score == target - 1)[0][0] + 1))
                else:
                    mrr.append(0)

    hit_rate = np.mean(hit) * 100
    mrr_rate = np.mean(mrr) * 100
    model.scheduler.step()
    return hit_rate, mrr_rate



In [ ]:
import torch
opt = type('', (), {})()
opt.hiddenSize = 100
opt.batchSize = 32
opt.nhead = 2
opt.layer = 2
opt.dropout = 0.1
opt.alpha = 0.5
opt.lr = 0.001
opt.l2 = 1e-5
opt.lr_dc_step = 3
opt.lr_dc = 0.1
opt.n_epochs = 10


# Model initialization
n_node = 37484
n_user = 1000
model = CollaborativeSelfAttentionNetwork(opt, n_node, n_user)
#model = trans_to_cuda(model)  # Make sure this function moves the model to GPU if available

# Generate user IDs (adjust if you have actual user IDs)
user_ids = np.random.randint(0, n_user, size=(len(train_loader.dataset),))

# Training loop
for epoch in range(opt.n_epochs):
    model.train()  # Set model to training mode
    total_loss = 0

    for batch in train_loader:
        items, session_ids, neighborhood_sessions, neighborhood_weights, targets = batch

        # Move data to GPU if applicable
        items = items.to(device)
        session_ids = session_ids.to(device)
        neighborhood_sessions = neighborhood_sessions.to(device)
        neighborhood_weights = neighborhood_weights.to(device)
        targets = targets.to(device)

        # Initialize optimizer
        optimizer = torch.optim.Adam(model.parameters(), lr=opt.lr, weight_decay=opt.l2)

        optimizer.zero_grad()  # Zero out gradients

        # Forward pass
        scores = model(items, session_ids, neighborhood_sessions, neighborhood_weights)

        # Compute loss (adjust according to your output layer)
        loss = model.compute_loss(scores, targets)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f'Epoch {epoch + 1}/{opt.n_epochs}, Loss: {avg_loss:.4f}')

    # Evaluate on test set after each epoch
    model.eval()  # Set model to evaluation mode
    hit_rate, mrr_rate = train_test(model, train_loader, test_loader, user_ids, neighborhood_data, device)
    print(f'Epoch {epoch + 1}/{opt.n_epochs}, Hit Rate: {hit_rate:.2f}%, MRR: {mrr_rate:.2f}%')

print("Training completed.")

RuntimeError: stack expects each tensor to be equal size, but got [6] at entry 0 and [2] at entry 1

In [ ]:
!pip freeze > requirements.txt